In [27]:
# Khai báo sử dụng các thư viện. Thực hiện chạy ngay trên CPU, do đó, chỉ thực hiện mô phỏng, nếu ổn định sẽ thực hiện chạy trên kaggle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

import regex as re
import os
import hashlib
from datetime import datetime
import hashlib
import copy

from tabulate import tabulate # Thư viện sử dụng hiển thị dữ liệu dạng bảng cho DataFrame

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [28]:
# Chỉnh sửa class Logcluster:
from Root_Drain import LogParser, Node, Logcluster            # Source code cũ của Drain

class Logcluster3(Logcluster):
    """
    Class được chỉnh sửa lại, bao gồm một số thông số như sau:
    
    Thuộc tính:
    ----------
        - `logTemplate`: Template đặc trưng đại diện cho nhóm log đó, ["The1", "the2", ...]
        - `logIDL`     : Danh sách các ID message log thuộc nhóm log trên, [1,2,3,4,5, ...] 
        - `levelL`     : Danh sách các Level (INFO, WARN, FATAL, ERROR) mà nhóm log có thể biểu diễn, ["WARN", "INFO", ...]
        - `idTemplate` : Mã định danh của template đó, có thể bị thay đổi do template.
        - `idLevelTemp`: Xác định mã định danh theo dạng {"INFO": "L1", ...} 
    """
    def __init__(self, logTemplate="", levelL=None, logIDL=None):
        self.logTemplate = logTemplate
        if logIDL is None:
            logIDL = []
        self.logIDL = logIDL
        self.levelL = list(levelL)
        self.idTemplate = None
        self.idLevelTemp = {"TEMP":None}
        
    def addIDLevel(self, logLevel):
        self.levelL.append(logLevel)



In [36]:
# Thực hiện chỉnh sửa lại 2 phương thức outputResult() và parser() của Drain:
# Kế thừa Drain bằng cách tạo ra class Drain2:
class LogParser2(LogParser):
    """
    Kế thừa từ LogParser được sử dụng để thực hiện sửa đổi lại sao cho phù hợp.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)       # Đọc thêm: https://viblo.asia/p/python-args-va-kwargs-gDVK2pdnlLj
        self.rootNode = Node()
        self.list_logClust = []
        self.list_eventsID = {}
        self.df_TemplateL = None
        self.list_counts = {"TEMP":0}
    
    def convert_unixtime(self, row):
        date_str = row["Date"]
        time_str = row["Time"]
        datetime_str = date_str[:2] + "-" + date_str[2:4] + "-" + date_str[4:] + " " + time_str[:2] + ":" + time_str[2:4] + ":" + time_str[4:]
        unixtime = datetime.strptime(datetime_str, "%m-%y-%d %H:%M:%S").timestamp()
        return int(unixtime)
    
    def outputResult(self, logClustL):
        """
        Thực hiện in thông tin kết quả ra ngoài sau khi hoàn thành quá trình phân tích
        
        Tham số:
        --------
            - `logClustL`   : Danh sách các nhóm log đã thu thập được trong quá trình phân tách
        """
        # ----------- Khai báo các biến sử dụng ----------#
        log_templates = [0] * self.df_log.shape[0]
        log_templateids = [0] * self.df_log.shape[0]
        list_events = []                            # Mảng list_events lưu giữ các giá trị template
        dictionary_events = self.list_eventsID      # Dictionary lưu giữ các đối tượng idTemplate và các thông số của nó: "idTemplate": {"TEMP":"L5", "INFO":"I2", ...}
    
        # --------------Thực hiện phân tích--------------#
        # * Duyệt qua từng đối tượng Logcluster để thêm các giá trị vào từng biến trên
        for logClust in logClustL:
            template_str = " ".join(logClust.logTemplate)
            occurrence = len(logClust.logIDL)
            template_id = hashlib.md5(template_str.encode("utf-8")).hexdigest()[0:8]
            logClust.idTemplate = template_id   # không cần cũng được
            for logID in logClust.logIDL:
                logID -= 1
                log_templates[logID] = template_str
                log_templateids[logID] = template_id
            logClust.logIDL=[]
            list_events.append([template_id, template_str, occurrence, logClust.levelL, logClust.idLevelTemp])
            if template_id not in dictionary_events:
                dictionary_events[template_id] = logClust.idLevelTemp

        # DataFrame df_event lưu giữ các thông tin của Template
        df_event = pd.DataFrame(
            list_events, columns=["EventId", "EventTemplate", "Occurrences", "logLevel", "IDlogLevel"]
        )
        # varL1.df_all_events = df_event
        
        # DataFrame df_log lưu giữ các thông tin của từng dòng thông điệp log
        self.df_log["EventId"] = log_templateids
        self.df_log["EventTemplate"] = log_templates
        
        # Chuyển đổi thời gian theo dạng UnixTime:
        # Tạo thêm cột "UnixTime" trong DataFrame: df_log
        self.df_log["UnixTime"] = self.df_log.apply(self.convert_unixtime, axis=1)
                
        if self.keep_para:
            self.df_log["ParameterList"] = self.df_log.apply(
                self.get_parameter_list, axis=1
            )
        self.df_log["IDLevel"] = self.df_log.apply(
            lambda row: dictionary_events.get(row["EventId"], {}).get(row["Level"], None),
            axis=1
        )
        self.df_log["LineId2"] = self.df_log.apply(
            lambda row: dictionary_events.get(row["EventId"], {}).get("TEMP", None),
            axis=1
        )
    
    def parse(self, logName):
        """
        Phương thức thực hiện quá trình phân tích log
        
        Tham số: 
        --------
            - `logName`: Tên file đầu vào thực hiện phân tích
        """
        print("File đầu vào: " + os.path.join(self.path, logName))
        start_time = datetime.now()
        self.logName = logName

        # ------Gán lại các đối tượng sử dụng--------#
        rootNode = self.rootNode       # Địa chỉ Node gốc
        logCluL = self.list_logClust   # Lưu giữ các đối tượng nhóm log Logcluster2 trước đó.
        templateLogCluL = []           # Lưu giữ các template đã có trước đó.

        # ----------------Load dữ liệu---------------#
        self.load_data() 

        #-----------------Phân tích------------------#
        # * Gán giá trị cho danh sách đếm template:
        unique_Level = self.df_log["Level"].unique()
        for value in unique_Level:
            if value not in self.list_counts:
                self.list_counts[value] = 0

        # * Thực hiện phân tích và lấy Template cho từng dòng thông điệp log
        count = 0
        for idx, line in self.df_log.iterrows(): 
            logLevel = str(line["Level"])
            logID = line["LineId"]
            logmessageL = self.preprocess(line["Content"]).strip().split()  
            matchCluster = self.treeSearch(rootNode, logmessageL)           
            if matchCluster is None:
                newCluster = Logcluster3(logTemplate=logmessageL, levelL=[logLevel], logIDL=[logID])
                
                # - Nếu có template mới, tạo ra, tăng biến đếm, gán id level template cho mẫu:
                self.list_counts["TEMP"] += 1
                self.list_counts[logLevel] += 1
                newCluster.idLevelTemp["TEMP"] = "L"+ str(self.list_counts["TEMP"])
                newCluster.idLevelTemp[logLevel] = logLevel[0] + str(self.list_counts[logLevel])
                
                # - Thêm chúng vào cây phân tích
                logCluL.append(newCluster)
                self.addSeqToPrefixTree(rootNode, newCluster)
            else:
                # - Nếu template đã có:
                newTemplate = self.getTemplate(logmessageL, matchCluster.logTemplate)
                matchCluster.logIDL.append(logID)
                # + Nếu có sự thay đổi trong template thành một template mới, cập nhật
                if " ".join(newTemplate) != " ".join(matchCluster.logTemplate):
                    matchCluster.logTemplate = newTemplate
                    
                # + Nếu có thêm một level mới biểu diễn template này, cập nhật nó
                if logLevel not in matchCluster.levelL:
                    matchCluster.addIDLevel(logLevel)
                    self.list_counts[logLevel] += 1
                    matchCluster.idLevelTemp[logLevel] = logLevel[0] + str(self.list_counts[logLevel])
                    
            count += 1
            if count % 100000 == 0 or count == len(self.df_log):
                print("Processed {0:.1f}% of log lines.".format(count * 100.0 / len(self.df_log))) # Chỉ sửa mỗi vị trí này.

        # if not os.path.exists(self.savePath):
        #     os.makedirs(self.savePath)
        self.outputResult(self.list_logClust)
        print("Phân tích hoàn thành! [Thời gian thực hiện: {!s}]".format(datetime.now() - start_time))


In [30]:
input_dir  = '../../Dataset/HDFS/v1/'   # Đường dẫn vào thư mục chứa file đầu vào
output_dir = 'Result/'                  # Thư mục kết quả
log_file   = 'test2.log'                 # Tên file đầu vào
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
# Biểu thức chính quy
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Ngưỡng tương đồng
depth      = 4  # Độ sâu của cây phân tích

parserObj = LogParser2(log_format, indir=input_dir, outdir=output_dir, depth=depth, st=st, rex=regex)
# parserObj.parse(log_file)

selected_columns = ['LineId', 'Level', 'EventId', 'EventTemplate', 'IDLevel', 'LineId2']
for i in range(1,6):
    str_file = "test"+str(i)+".log"
    parserObj.parse(str_file)
    # outputResult2(parserObj.list_logClust)
    parserObj.df_log.loc[:, selected_columns].head(60)
    parserObj.df_log = pd.DataFrame()

In [31]:
# Chỉnh sửa lại phương thức parser trong LogParser, chỉnh sửa trực tiếp
def parser22(log_f):
    log_file=log_f
    logName = log_file

    print("File đầu vào: " + os.path.join(parserObj.path, logName))
    start_time = datetime.now()
    parserObj.logName = logName

    # ------Gán lại các đối tượng sử dụng--------#
    rootNode = parserObj.rootNode       # Địa chỉ Node gốc
    logCluL = parserObj.list_logClust   # Lưu giữ các đối tượng nhóm log Logcluster2 trước đó.
    templateLogCluL = []                # Lưu giữ các template đã có trước đó.

    # ----------------Load dữ liệu---------------#
    parserObj.load_data() 

    #-----------------Phân tích------------------#
    # * Gán giá trị cho danh sách đếm template:
    unique_Level = parserObj.df_log["Level"].unique()
    for value in unique_Level:
        if value not in parserObj.list_counts:
            parserObj.list_counts[value] = 0

    # * Thực hiện phân tích và lấy Template cho từng dòng thông điệp log
    count = 0
    for idx, line in parserObj.df_log.iterrows(): 
        logLevel = str(line["Level"])
        logID = line["LineId"]
        logmessageL = parserObj.preprocess(line["Content"]).strip().split()  
        matchCluster = parserObj.treeSearch(rootNode, logmessageL)           
        if matchCluster is None:
            newCluster = Logcluster3(logTemplate=logmessageL, levelL=[logLevel], logIDL=[logID])
            
            # - Nếu có template mới, tạo ra, tăng biến đếm, gán id level template cho mẫu:
            parserObj.list_counts["TEMP"] += 1
            parserObj.list_counts[logLevel] += 1
            newCluster.idLevelTemp["TEMP"] = "L"+ str(parserObj.list_counts["TEMP"])
            newCluster.idLevelTemp[logLevel] = logLevel[0] + str(parserObj.list_counts[logLevel])
            
            # - Thêm chúng vào cây phân tích
            logCluL.append(newCluster)
            parserObj.addSeqToPrefixTree(rootNode, newCluster)
        else:
            # - Nếu template đã có:
            newTemplate = parserObj.getTemplate(logmessageL, matchCluster.logTemplate)
            matchCluster.logIDL.append(logID)
            # + Nếu có sự thay đổi trong template thành một template mới, cập nhật
            if " ".join(newTemplate) != " ".join(matchCluster.logTemplate):
                matchCluster.logTemplate = newTemplate
                
            # + Nếu có thêm một level mới biểu diễn template này, cập nhật nó
            if logLevel not in matchCluster.levelL:
                matchCluster.addIDLevel(logLevel)
                parserObj.list_counts[logLevel] += 1
                matchCluster.idLevelTemp[logLevel] = logLevel[0] + str(parserObj.list_counts[logLevel])
                
        count += 1
        if count % 100000 == 0 or count == len(parserObj.df_log):
            print("Processed {0:.1f}% of log lines.".format(count * 100.0 / len(parserObj.df_log))) # Chỉ sửa mỗi vị trí này.
    # parserObj.list_logClust = copy.deepcopy(logCluL)

    # if not os.path.exists(self.savePath):
    #     os.makedirs(self.savePath)
    # self.outputResult(logCluL)
    print("Phân tích hoàn thành! [Thời gian thực hiện: {!s}]".format(datetime.now() - start_time))

# for i in range(1,6):
#     str_file = "test"+str(i)+".log"
#     parser22(str_file)

##### Thực hiện tạo ra một dictionary như sau:
##### - {"idTamplate": "L1", ...}

In [35]:
# Thực hiện sửa lại phương thức outputResult, viết trực tiếp để dễ chỉnh sửa:
# Class này chỉ để lưu thông tin.
class varL:
    def __init__(self):
        df_all_events = pd.DataFrame()
        df_all_events2 = pd.DataFrame()

varL1 = varL()


# def levelTempID(row):
#     return dictionary_events[row["EventId"]][row["Level"]]
# def alternateTempID(row):
#     return dictionary_events[row["EventId"]]["TEMP"]

def outputResult2(logClustL):
    # ----------- Khai báo các biến sử dụng ----------#
    log_templates = [0] * parserObj.df_log.shape[0]
    log_templateids = [0] * parserObj.df_log.shape[0]
    list_events = []  # Mảng list_events lưu giữ các giá trị template
    dictionary_events = parserObj.list_eventsID   # Dictionary lưu giữ các đối tượng idTemplate và các thông số của nó: "idTemplate": {"TEMP":"L5", "INFO":"I2", ...}
   
    # --------------Thực hiện phân tích--------------#
    # * Duyệt qua từng đối tượng Logcluster để thêm các giá trị vào từng biến trên
    for logClust in logClustL:
        template_str = " ".join(logClust.logTemplate)
        occurrence = len(logClust.logIDL)
        template_id = hashlib.md5(template_str.encode("utf-8")).hexdigest()[0:8]
        logClust.idTemplate = template_id   # không cần cũng được
        for logID in logClust.logIDL:
            logID -= 1
            log_templates[logID] = template_str
            log_templateids[logID] = template_id
        logClust.logIDL=[]
        list_events.append([template_id, template_str, occurrence, logClust.levelL, logClust.idLevelTemp])
        if template_id not in dictionary_events:
            dictionary_events[template_id] = logClust.idLevelTemp

    # DataFrame df_event lưu giữ các thông tin của Template
    df_event = pd.DataFrame(
        list_events, columns=["EventId", "EventTemplate", "Occurrences", "logLevel", "IDlogLevel"]
    )
    varL1.df_all_events = df_event
    
    # DataFrame df_log lưu giữ các thông tin của từng dòng thông điệp log
    parserObj.df_log["EventId"] = log_templateids
    parserObj.df_log["EventTemplate"] = log_templates
            
    if parserObj.keep_para:
        parserObj.df_log["ParameterList"] = parserObj.df_log.apply(
            parserObj.get_parameter_list, axis=1
        )
    parserObj.df_log["IDLevel"] = parserObj.df_log.apply(
        lambda row: dictionary_events.get(row["EventId"], {}).get(row["Level"], None),
        axis=1
    )
    parserObj.df_log["LineId2"] = parserObj.df_log.apply(
        lambda row: dictionary_events.get(row["EventId"], {}).get("TEMP", None),
        axis=1
    )
# varL1.df_all_events.head(30)
selected_columns = ['LineId', 'Level', 'EventId', 'EventTemplate', 'IDLevel', 'LineId2']
for i in range(1,6):
    str_file = "test"+str(i)+".log"
    parser22(str_file)
    outputResult2(parserObj.list_logClust)
    parserObj.df_log.loc[:, selected_columns].head(60)
    parserObj.df_log = pd.DataFrame()

# selected_columns = ['LineId', 'Level', 'EventId', 'EventTemplate', 'IDLevel', 'LineId2']
# print(parserObj.df_log.columns)
# parserObj.df_log.loc[:, selected_columns].head(60)

File đầu vào: ../../Dataset/HDFS/v1/test1.log
Total lines:  200500
Processed 49.9% of log lines.
Processed 99.8% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:00:13.995417]


,LineId,Level,EventId,EventTemplate,IDLevel,LineId2
0,1,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
1,2,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2
2,3,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
3,4,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
4,5,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3
5,6,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3
6,7,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4
7,8,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4
8,9,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3
9,10,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4


File đầu vào: ../../Dataset/HDFS/v1/test2.log
Total lines:  397000
Processed 25.2% of log lines.
Processed 50.4% of log lines.
Processed 75.6% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:00:28.229747]


,LineId,Level,EventId,EventTemplate,IDLevel,LineId2
0,1,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
1,2,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
2,3,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
3,4,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
4,5,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
5,6,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
6,7,INFO,32777b38,Verification succeeded for <*>,I13,L13
7,8,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5
8,9,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2
9,10,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5


File đầu vào: ../../Dataset/HDFS/v1/test3.log
Total lines:  200500
Processed 49.9% of log lines.
Processed 99.8% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:00:13.852110]


,LineId,Level,EventId,EventTemplate,IDLevel,LineId2
0,1,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3
1,2,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4
2,3,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
3,4,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
4,5,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
5,6,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3
6,7,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4
7,8,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3
8,9,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4
9,10,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1


File đầu vào: ../../Dataset/HDFS/v1/test4.log
Total lines:  200500
Processed 49.9% of log lines.
Processed 99.8% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:00:14.309842]


,LineId,Level,EventId,EventTemplate,IDLevel,LineId2
0,1,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
1,2,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
2,3,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
3,4,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
4,5,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
5,6,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
6,7,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
7,8,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
8,9,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35
9,10,INFO,626085d5,<*> Served block <*> to <*>,I10,L10


File đầu vào: ../../Dataset/HDFS/v1/test5.log
Total lines:  198999
Processed 50.3% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:00:14.086164]


,LineId,Level,EventId,EventTemplate,IDLevel,LineId2
0,1,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5
1,2,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2
2,3,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2
3,4,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5
4,5,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5
5,6,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5
6,7,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5
7,8,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2
8,9,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2
9,10,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1
